In [33]:
# install psycopg2 module to support SQL Alchemy and postgresql

"""
SQL Queries to be run after ingestion of data to define point geometry objects 
based on lat/lon of data for use with Geoserver, helps Geoserver define 
bounding boxes and such

ALTER TABLE public.active_storms ADD COLUMN geom geometry(Point, 4326);
ALTER TABLE public.historical_storms ADD COLUMN geom geometry(Point, 4326);

UPDATE public.active_storms SET geom = ST_SetSRID(ST_MakePoint("LON", "LAT"), 4326);
UPDATE public.historical_storms SET geom = ST_SetSRID(ST_MakePoint("LON", "LAT"), 4326);
"""
import os
from dotenv import load_dotenv
import pandas as pd
import psycopg2
from sqlalchemy import create_engine, text
from pathlib import Path

load_dotenv()

pg_host = os.getenv('PG_HOST')
pg_port = int(os.getenv('PG_PORT'))
pg_user = os.getenv('PG_USER')
pg_pass = os.getenv('PG_PASS')
pg_db = os.getenv('PG_DB')

pg_ibtracs_historical_table = os.getenv('PG_IBTRACS_HISTORICAL_TABLE')
pg_ibtracs_active_table = os.getenv('PG_IBTRACS_ACTIVE_TABLE')
pg_eccc_table = os.getenv('PG_ECCC_ACTIVE_TABLE')

ibtracs_active_file = Path(os.getenv('ACTIVE_STORMS_CSV'))
ibtracs_historical_file = Path(os.getenv('HISTORICAL_STORMS_CSV'))

ibtracs_active_schema = Path(os.getenv('IBTRACS_ACTIVE_SCHEMA'))
ibtracs_historical_schema = Path(os.getenv('IBTRACS_HISTORICAL_SCHEMA'))

# Tells pandas to skip the 2nd line in the CSV file that specifies unit types 
# for the columns - this row doesn't need to be inserted into the postgis table
skip_rows = [1]

# Empty values in the CSV file are represented by an empty string, this value 
# isn't accounted for by default in pandas, so it needs to be specified here, 
# a dictionary of values is also possible if there are other values that 
# should be interpreted as null or n/a
na_values = ' '

# Derived from postgres table structure
table_dtypes = {
    "SID": 'string', 
    "SEASON": 'Float32', 
    "NUMBER": 'Float32', 
    "BASIN": 'string', 
    "SUBBASIN": 'string', 
    "NAME": 'string', 
    "ISO_TIME": 'string', 
    "NATURE": 'string', 
    "LAT": 'Float32', 
    "LON": 'Float32', 
    "WMO_WIND": 'Float32', 
    "WMO_PRES": 'Float32', 
    "WMO_AGENCY": 'string', 
    "TRACK_TYPE": 'string', 
    "DIST2LAND": 'Float32', 
    "LANDFALL": 'Float32', 
    "IFLAG": 'string', 
    "USA_AGENCY": 'string', 
    "USA_ATCF_ID": 'string', 
    "USA_LAT": 'Float32', 
    "USA_LON": 'Float32', 
    "USA_RECORD": 'string', 
    "USA_STATUS": 'string', 
    "USA_WIND": 'Float32', 
    "USA_PRES": 'Float32', 
    "USA_SSHS": 'Float32', 
    "USA_R34_NE": 'Float32', 
    "USA_R34_SE": 'Float32', 
    "USA_R34_SW": 'Float32', 
    "USA_R34_NW": 'Float32', 
    "USA_R50_NE": 'Float32', 
    "USA_R50_SE": 'Float32', 
    "USA_R50_SW": 'Float32', 
    "USA_R50_NW": 'Float32', 
    "USA_R64_NE": 'Float32', 
    "USA_R64_SE": 'Float32', 
    "USA_R64_SW": 'Float32', 
    "USA_R64_NW": 'Float32', 
    "USA_POCI": 'Float32', 
    "USA_ROCI": 'Float32', 
    "USA_RMW": 'Float32', 
    "USA_EYE": 'Float32', 
    "TOKYO_LAT": 'Float32', 
    "TOKYO_LON": 'Float32', 
    "TOKYO_GRADE": 'Float32', 
    "TOKYO_WIND": 'Float32', 
    "TOKYO_PRES": 'Float32', 
    "TOKYO_R50_DIR": 'Float32', 
    "TOKYO_R50_LONG": 'Float32', 
    "TOKYO_R50_SHORT": 'Float32', 
    "TOKYO_R30_DIR": 'Float32', 
    "TOKYO_R30_LONG": 'Float32', 
    "TOKYO_R30_SHORT": 'Float32', 
    "TOKYO_LAND": 'Float32', 
    "CMA_LAT": 'Float32', 
    "CMA_LON": 'Float32', 
    "CMA_CAT": 'Float32', 
    "CMA_WIND": 'Float32', 
    "CMA_PRES": 'Float32', 
    "HKO_LAT": 'Float32', 
    "HKO_LON": 'Float32', 
    "HKO_CAT": 'string', 
    "HKO_WIND": 'Float32', 
    "HKO_PRES": 'Float32', 
    "NEWDELHI_LAT": 'Float32', 
    "NEWDELHI_LON": 'Float32', 
    "NEWDELHI_GRADE": 'string', 
    "NEWDELHI_WIND": 'Float32', 
    "NEWDELHI_PRES": 'Float32', 
    "NEWDELHI_CI": 'Float32', 
    "NEWDELHI_DP": 'Float32', 
    "NEWDELHI_POCI": 'Float32', 
    "REUNION_LAT": 'Float32', 
    "REUNION_LON": 'Float32', 
    "REUNION_TYPE": 'string', 
    "REUNION_WIND": 'Float32', 
    "REUNION_PRES": 'Float32', 
    "REUNION_TNUM": 'string', 
    "REUNION_CI": 'string', 
    "REUNION_RMW": 'Float32', 
    "REUNION_R34_NE": 'Float32', 
    "REUNION_R34_SE": 'Float32', 
    "REUNION_R34_SW": 'Float32', 
    "REUNION_R34_NW": 'Float32', 
    "REUNION_R50_NE": 'Float32', 
    "REUNION_R50_SE": 'Float32', 
    "REUNION_R50_SW": 'Float32', 
    "REUNION_R50_NW": 'Float32', 
    "REUNION_R64_NE": 'Float32', 
    "REUNION_R64_SE": 'Float32', 
    "REUNION_R64_SW": 'Float32', 
    "REUNION_R64_NW": 'Float32', 
    "BOM_LAT": 'Float32', 
    "BOM_LON": 'Float32', 
    "BOM_TYPE": 'string', 
    "BOM_WIND": 'Float32', 
    "BOM_PRES": 'Float32', 
    "BOM_TNUM": 'string', 
    "BOM_CI": 'string', 
    "BOM_RMW": 'Float32', 
    "BOM_R34_NE": 'Float32', 
    "BOM_R34_SE": 'Float32', 
    "BOM_R34_SW": 'Float32', 
    "BOM_R34_NW": 'Float32', 
    "BOM_R50_NE": 'Float32', 
    "BOM_R50_SE": 'Float32', 
    "BOM_R50_SW": 'Float32', 
    "BOM_R50_NW": 'Float32', 
    "BOM_R64_NE": 'Float32', 
    "BOM_R64_SE": 'Float32', 
    "BOM_R64_SW": 'Float32', 
    "BOM_R64_NW": 'Float32', 
    "BOM_ROCI": 'Float32', 
    "BOM_POCI": 'Float32', 
    "BOM_EYE": 'Float32', 
    "BOM_POS_METHOD": 'string', 
    "BOM_PRES_METHOD": 'string', 
    "NADI_LAT": 'Float32', 
    "NADI_LON": 'Float32', 
    "NADI_CAT": 'Float32', 
    "NADI_WIND": 'Float32', 
    "NADI_PRES": 'Float32', 
    "WELLINGTON_LAT": 'Float32', 
    "WELLINGTON_LON": 'Float32', 
    "WELLINGTON_WIND": 'Float32', 
    "WELLINGTON_PRES": 'Float32', 
    "DS824_LAT": 'Float32', 
    "DS824_LON": 'Float32', 
    "DS824_STAGE": 'string', 
    "DS824_WIND": 'Float32', 
    "DS824_PRES": 'Float32', 
    "TD9636_LAT": 'Float32', 
    "TD9636_LON": 'Float32', 
    "TD9636_STAGE": 'Float32', 
    "TD9636_WIND": 'Float32', 
    "TD9636_PRES": 'Float32', 
    "TD9635_LAT": 'Float32', 
    "TD9635_LON": 'Float32', 
    "TD9635_WIND": 'Float32', 
    "TD9635_PRES": 'Float32', 
    "TD9635_ROCI": 'Float32', 
    "NEUMANN_LAT": 'Float32', 
    "NEUMANN_LON": 'Float32', 
    "NEUMANN_CLASS": 'string', 
    "NEUMANN_WIND": 'Float32', 
    "NEUMANN_PRES": 'Float32', 
    "MLC_LAT": 'Float32', 
    "MLC_LON": 'Float32', 
    "MLC_CLASS": 'string', 
    "MLC_WIND": 'Float32', 
    "MLC_PRES": 'Float32', 
    "USA_GUST": 'Float32', 
    "BOM_GUST": 'Float32', 
    "BOM_GUST_PER": 'Float32', 
    "REUNION_GUST": 'Float32', 
    "REUNION_GUST_PER": 'Float32', 
    "USA_SEAHGT": 'Float32', 
    "USA_SEARAD_NE": 'Float32', 
    "USA_SEARAD_SE": 'Float32', 
    "USA_SEARAD_SW": 'Float32', 
    "USA_SEARAD_NW": 'Float32', 
    "STORM_SPEED": 'Float32', 
    "STORM_DIR": 'Float32'
}

# eccc_source_path = Path(os.getenv('ECCC_SHP_SOURCE'))

engine = create_engine(f"postgresql+psycopg2://{pg_user}:{pg_pass}@{pg_host}:{pg_port}/{pg_db}")

In [34]:
def process_ibtracs(source_csv_file, destination_table, pg_engine, table_schema):
    df = pd.read_csv(filepath_or_buffer=source_csv_file, header=0, skiprows=skip_rows, parse_dates=True, dtype=table_dtypes, na_values=na_values)
    
    print(df.info(verbose=True))
    print(df.head())

    with pg_engine.begin() as pg_conn:
        # Create Tables (if not exist using schemas)
        # Generate shapes for IBTRACS?
        print("Creating Table (if not exists)...")
        sql = Path(table_schema).read_text()
        print(sql)
        pg_conn.execute(text(sql))
        
        # truncate tables
        sql = f"DELETE FROM {destination_table};"
        print(sql)
        pg_conn.execute(text(sql))

        pg_conn.execute(text("COMMIT;"))
    
    # populate table
    print("Populating Table...")
    df.to_sql(destination_table, pg_engine, chunksize=1000, method='multi', if_exists='append', index=False, schema='public')
    
    with pg_engine.begin() as pg_conn:
        # add lat/long geometry points
        print("Adding Geometry Column...")
        sql = f'ALTER TABLE public.{destination_table} ADD COLUMN geom geometry(Point, 4326);'
        print(sql)
        pg_conn.execute(text(sql))

        print("Updating Geometry...")
        sql = f'UPDATE public.{destination_table} SET geom = ST_SetSRID(ST_MakePoint("LON", "LAT"), 4326);'
        print(sql)
        pg_conn.execute(text(sql))

        pg_conn.execute(text("COMMIT;"))
        print("Fin.")


In [35]:
print("Processing Active Storms...")
process_ibtracs(source_csv_file=ibtracs_active_file, destination_table=pg_ibtracs_active_table, pg_engine=engine, table_schema=ibtracs_active_schema)


Processing Active Storms...
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 180 entries, 0 to 179
Data columns (total 163 columns):
 #    Column            Dtype  
---   ------            -----  
 0    SID               string 
 1    SEASON            Float32
 2    NUMBER            Float32
 3    BASIN             string 
 4    SUBBASIN          string 
 5    NAME              string 
 6    ISO_TIME          string 
 7    NATURE            string 
 8    LAT               Float32
 9    LON               Float32
 10   WMO_WIND          Float32
 11   WMO_PRES          Float32
 12   WMO_AGENCY        string 
 13   TRACK_TYPE        string 
 14   DIST2LAND         Float32
 15   LANDFALL          Float32
 16   IFLAG             string 
 17   USA_AGENCY        string 
 18   USA_ATCF_ID       string 
 19   USA_LAT           Float32
 20   USA_LON           Float32
 21   USA_RECORD        string 
 22   USA_STATUS        string 
 23   USA_WIND          Float32
 24   USA_PRES          Float32
 2

IntegrityError: (psycopg2.errors.NotNullViolation) null value in column "BASIN" of relation "ibtracs_active_storms" violates not-null constraint
DETAIL:  Failing row contains (2023151N25272, 2023, 19, null, GM, ARLENE, 2023-05-30 18:00:00, NR, 24.8000, -88.3000, null, null, null, PROVISIONAL, 368, 368, O_____________, nhc_working_bt, AL022023, 24.8000, -88.3000, null, DB, 15, null, -3, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null,

In [ ]:
print("Processing Historical Storms...")
process_ibtracs(source_csv_file=ibtracs_historical_file, destination_table=pg_ibtracs_historical_table, pg_engine=engine, table_schema=ibtracs_historical_schema)


In [ ]:
print("End.")
